# Домашнее задание 2
## Named Entity Recognition and Event Extraction

deadline: 06 ноября 2022, 23:59

В этом домашнем задании вы будете работать с двумя корпусами - LitBank и MultiNERD_EN.
Первый корпус (LitBank) собран из популярных художественных произведений на английском языке и содержит разметку по именованным сущностям и событиям, корпус состоит из 100 текстов по примерно 2000 слов каждый. 

Корпус описан в статьях:
* David Bamman, Sejal Popat, Sheng Shen, An Annotated Dataset of Literary Entities http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/naacl2019_literary_entities.pdf
* Matthew Sims, Jong Ho Park, David Bamman, Literary Event Detection,  http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/acl2019_literary_events.pdf

Корпус доступен в репозитории проекта:  https://github.com/dbamman/litbank

Статья и код, использованный для извлечения именованных сущностей: 
* Meizhi Ju, Makoto Miwa and Sophia Ananiadou, A Neural Layered Model for Nested Named Entity Recognition, https://github.com/meizhiju/layered-bilstm-crf

Структура корпуса устроена так. 
Первый уровень: 
* entities -- разметка по сущностям
* events -- разметка по сущностям


В корпусе используются 6 типов именованных сущностей: PER, LOC, ORG, FAC, GPE, VEH (имена, локации, организации, помещения, топонимы, средства перемещения), допускаются вложенные сущности. 

События выражается одним словом - *триггером*, которое может быть глаголом, прилагательным и существительным. В корпусе описаны события, которые действительно происходят и не имеют гипотетического характера. 
Пример: she *walked* rapidly and resolutely, здесь *walked* -- триггер события. Типы событий не заданы. 


Второй уровень:
* brat -- рабочие файлы инструмента разметки brat, ann-файлы содержат разметку, txt-файлы – сырые тексты 
* tsv -- tsv-файлы содержат разметку в IOB формате,

Второй корпус (MultiNERD_EN) состоит из 164 тысяч предложений по различным тематикам. От предыдущего корпуса он отличается большим числом (15) типов именованных сущностей. Также одной из сущностей является сущность-событие (EVE). Каждая сущность может состоять как из одного, так и нескольких слов.

Корпус доступен в репозитории проекта:  https://github.com/Babelscape/multinerd

В статьях и репозиториях вы найдете идеи, которые помогут вам выполнить домашнее задание. Их стоит воспринимать как руководство к действию, и не стоит их копировать и использовать повторно. Обученные модели использовать не нужно, код для их обучения можно использовать как подсказку. 

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 4-х человек.
2. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо в ipython-тетрадке. При этом прикладывать ipynb-файл обязательно. 
43. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
4. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
5. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
6. Плагиат и любое недобросовестное цитирование приводит к обнулению оценки. 


## Часть 1. [2 балла] Эксплоративный анализ 
1. [0.4 балла] Найдите топ 10 (по частоте) именованных сущностей каждого из типов для каждого корпуса. Произведите сравнение частотности по совпадающим сущностям при сопоставлении корпусов друг с другом.
2. [0.4 балла] Найдите топ 10 (по частоте) частотных триггеров событий (на основе LitBank). Произведите анализ из пункта 1 для именованной сущности типа EVE (на основе MultiNERD_EN). 
3. [0.6 балла] Кластеризуйте все уникальные триггеры событий (LitBank), используя эмбеддинги слов и любой алгоритм кластеризации (например, аггломеративный иерархический алгоритм кластеризации) и попробуйте проинтерпретировать кластеры: есть ли очевидные типы событий? 
4. [0.6 балла] Отдельно произведите анализ из п. 3 для сущностей типа EVE (на основе MultiNERD_EN). Сравните результаты из п. 3 и п. 4.

[бонус] Визуализируйте полученные кластеры с помощью TSNE или UMAP

[бонус] Постройте тематическую модель по корпусу LitBank и сравните кластеры триггеров и выделенные темы: есть ли схожие паттерны в тематической модели и в структуре кластеров?

В следующих частях домашнего задания вам понадобится train-test-dev разбиения. Авторы статей предлагают следующую структуру разбиения для корпуса LitBank: обучающее множество – 80 книг, валидационное – 10 книг, тестовое – 10 книг. Предложения из одного источника не должны попадать в разные сегменты разбиения.
Для корпуса MultiNERD_EN произведите стратифицированное разбиение по предложениям в соотношении 80%:10%:10%. Стратификацию производить в отношении пропорции по именованным сущностям (понятно, что с учётом всех вводных идеального разбиения не получится, но старайтесь сохранить распределения по типам сущностей для каждой и подвыборок).


## Часть 2. [3 балла] Извлечение именованных сущностей
Для каждой корпуса необходимо осуществить раздельно следующую процедуру:
1. [0.75 балла] Обучите стандартную модель для извлечения именованных сущностей, CNN-BiLSTM-CRF, для извлечения именованных *низкоуровневых именованных сущностей*, т.е. для самых коротких из вложенных сущностей. 
Модель устроена так: сверточная сеть на символах + эмбеддинги слов + двунаправленная LSTM сеть (модель последовательности) + CRF (глобальная нормализация).
2. [0.75 балла] Замените часть модели на символах и словах (CNN + эмбеддинги словах) на ELMo и / или BERT. Должна получиться модель ELMo / BERT + BiLSTM + CRF. 
3. [1.5 балла] Замените модель последовательности (BiLSTM) на другой слой, например, на Transformer. Должна получиться модель CNN  + Transformer + CRF.
В результате у вас будет 2 набора моделей - проверьте качество извлечения на тестовых данных.
- Примените модели первого корпуса ко всем (!!!) предложениям второго корпуса и наоборот.
- Составьте отчёт по качеству работы моделей в терминах извлечения по типам сущностей, которые встречаются в обоих корпусах. Метрику выберите самостоятельно.
- Проанализируйте ошибки моделей в терминах предсказания сущностям первого корпуса специфических типов для второго корпуса. И наоборот - когда специфическим типам второго корпуса модель, обученная на LitBank, прсваивает одну из своих категорий.

[бонус] Дообучите BERT для извлечения именованных сущностей.

[бонус] Используйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018]

[бонус] Модифицируйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018]: вместо эмбеддингов слов используйте ELMo и/или BERT. 

## Часть 3. [2 балла] Извлечение событий 

1. [0.75 балла] Используйте BiLSTM на эмбеддингах слов для извлечения триггеров событий (на основе LitBank). 

2. [0.75 балла] Замените часть модели на словах на ELMo и/или BERT. Должна получиться модель ELMo / BERT + BiLSTM.

3. [0.5 балла] Проверьте "извлекающую" силу модели на данных MultiNERD_EN для событий типа EVE. 

[бонус] Предобучите BiLSTM как языковую модель. Дообучите ее для извлечения триггеров. 

[бонус] Дообучите BERT для извлечения триггеров событий. 

## Часть 4. [2 балла] Одновременное извлечение именованных сущностей и событий 
1. [0.75 балла] Обучите модель для совместного извлечения именованных сущностей и триггеров событий только на основе LitBank (!). У модели должен быть общий энкодер (например, CNN + BiLSMT, ELMo + BiLSTM, BERT + BiLSTM) и два декодера: один отвечает за извлечение именованных сущностей, другой отвечает за извлечение триггеров событий.

2. [0.75 балла] Создайте единую обучающую подвыборку, единую валидационную и единую тестовую на основе разбиений, которые вы произвели в первой части и обучите модель. (Единую - то есть на основе обоих корпусов сразу)

3. [0.5 балла] Сравните предсказательную силу модели из п.1 и модели из п.2 как на совместных подвыборках, так и на раздельных. Проанализируйте полученный результат. Приводит ли обогащение дополнительными данными к улучшению "извлекающей" способности модели?

[бонус] Добавьте в модель механизм внимания, таким способом, который покажется вам разумным.

[бонус] Визуализируйте карты механизма внимания (attention). 

## Часть 5. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?